In [2]:
import pandas as pd
import numpy as np
import pylab as plt
import glob
import os
import random
import keras
from keras import backend as K
from keras import layers
from keras import metrics
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.models import Sequential, Model
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import cv2
from tqdm import tqdm
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from collections import Counter
from sklearn.model_selection import train_test_split
import sklearn
from collections import Counter
from pandas import DataFrame
import gc
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight

In [3]:
def load_data_local(new_w,new_h):
    train_labels = pd.read_csv(r'C:\Users\aleja\OneDrive\Desktop\Kaggle\Leave Issues Competition\train.csv',sep=',')
    path_loop = r"C:\Users\aleja\OneDrive\Desktop\Kaggle\Leave Issues Competition\train_images\*.*"
    path = r"C:\Users\aleja\OneDrive\Desktop\Kaggle\Leave Issues Competition\train_images"
    onlyfiles = next(os.walk(path))[2] #dir is your directory path as string
    numOfFiles = len(onlyfiles)
    data = []
    for file in tqdm(glob.glob(path_loop)):
        a=cv2.imread(file)
        name_file = os.path.basename(file)
        label = train_labels.loc[train_labels['image_id'] == name_file,'label'].values
        #conversion numpy array into rgb image to show
        c = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        h, w, channels = c.shape
        #input size of Resnet architecture
        frame_rgb = cv2.resize(c,(new_w,new_h),interpolation=cv2.INTER_CUBIC)
        data.append([frame_rgb,label])
    return data

In [4]:
def load_data(new_w,new_h):
    train_labels = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv', sep=',')
    path = '../input/cassava-leaf-disease-classification/train_images'
    path_loop = r'../input/cassava-leaf-disease-classification/train_images/*.*'
    onlyfiles = next(os.walk(path))[2] #dir is your directory path as string
    numOfFiles = len(onlyfiles)
    data = []
    for file in tqdm(glob.glob(path_loop)):
        a=cv2.imread(file)
        name_file = os.path.basename(file)
        label = train_labels.loc[train_labels['image_id'] == name_file,'label'].values
        #conversion numpy array into rgb image to show
        c = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        h, w, channels = c.shape
        #input size of Resnet architecture
        frame_rgb = cv2.resize(c,(new_w,new_h),interpolation=cv2.INTER_CUBIC)
        data.append([frame_rgb,label])
    return data


In [5]:
#Change before submitting to Kaggle from load_data_local to load_data
IMG_SIZE_X = 128
IMG_SIZE_Y = 128
data = load_data(IMG_SIZE_X, IMG_SIZE_Y)

100%|██████████| 21397/21397 [06:53<00:00, 51.75it/s]


In [6]:
def train_test_split_modified(data):
    x_data = []
    y_data = []
    
    for feature, label in data:
        x_data.append(feature)
        y_data.append(label)
        
    X_data, X_test, Y_data, Y_test = sklearn.model_selection.train_test_split(x_data, y_data, stratify = y_data, test_size=0.1, random_state=42)
    X_model_1, X_aux, Y_model_1, Y_aux = sklearn.model_selection.train_test_split(X_data, Y_data, stratify = Y_data, test_size=0.66, random_state=42)
    X_model_2, X_model_3, Y_model_2, Y_model_3 = sklearn.model_selection.train_test_split(X_aux, Y_aux, stratify = Y_aux, test_size=0.5, random_state=42)
    return X_test, Y_test, X_model_1, Y_model_1, X_model_2, Y_model_2, X_model_3, Y_model_3
    


In [7]:
def data_preparation(X_data, Y_data, new_h, new_w, channels):
    x_train,x_val,y_train,y_val = train_test_split(X_data,Y_data,stratify=Y_data, train_size=0.8, random_state=42)
    x_train = np.array(x_train) / 255
    x_val = np.array(x_val) / 255
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    x_train = x_train.reshape(-1, new_h, new_w, channels)
    x_val = x_val.reshape(-1, new_h, new_w, channels)
    y_train = np.concatenate(y_train, axis=0)
    y_val = np.concatenate(y_val, axis=0)
    return x_train, x_val, y_train, y_val

In [39]:
#Encode target variables
def encode_target_variable(y_train,y_val):

    class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)
    d_class_weights = dict(enumerate(class_weights))
    print(d_class_weights)
    onehot_encoder = OneHotEncoder(sparse=False)
    y_train = y_train.reshape(-1,1)
    y_val = y_val.reshape(-1,1)
    y_train_onehot = onehot_encoder.fit_transform(y_train)
    y_val_onehot = onehot_encoder.fit_transform(y_val)
    return y_train_onehot, y_val_onehot, d_class_weights

In [9]:
def create_model_0(input_shape):
    input_img = Input(shape=input_shape)  
    x = Conv2D(16, (5, 5), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (5, 5), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (5, 5), activation='relu', padding='same')(x)
    x = Dropout(0.3)(x)
    y = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    y = MaxPooling2D((2, 2), padding='same')(y)
    y = Conv2D(16, (3, 3), activation='relu', padding='same')(y)
    y = MaxPooling2D((2, 2), padding='same')(y)
    y = Conv2D(16, (3, 3), activation='relu', padding='same')(y)
    y = Dropout(0.4)(y)
    z = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    z = MaxPooling2D((2, 2), padding='same')(z)
    z = Conv2D(16, (3, 3), activation='relu', padding='same')(z)
    z = MaxPooling2D((2, 2), padding='same')(z)
    z = Conv2D(16, (3, 3), activation='relu', padding='same')(z)

    out = layers.concatenate([x, y, z])
    out = layers.Flatten()(out)
    out = Dense(16, activation='selu')(out)
    out = Dense(5, activation='softmax')(out)
    
    model_f = Model(inputs=[input_img], outputs=[out])
    model_f.summary()
    return model_f

In [38]:
def create_model_1(x_train,x_val,input_shape):
    weights_path = '../input/weightsclimbers/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    resnet50 = ResNet50(weights = None ,include_top=False, input_shape=(IMG_SIZE_X, IMG_SIZE_Y,3))
    resnet50.load_weights(weights_path, by_name=True)
    resnet50.trainable = False # remove if you want to retrain resnet weights
    # resnet50.summary()
    transfer_model_1 = Sequential() 
    transfer_model_1.add(resnet50)
    transfer_model_1.add(Flatten())
    transfer_model_1.add(Dense(128, activation='relu'))
    transfer_model_1.add(Dropout(0.2))
    transfer_model_1.add(Dense(5, activation='softmax'))
    transfer_model_1.summary()
    x_train_new = x_train
    x_val_new = x_val
    x_train_new = keras.applications.resnet50.preprocess_input(x_train_new)
    x_val_new = keras.applications.resnet50.preprocess_input(x_val_new)
    return transfer_model_1, x_train_new, x_val_new

In [37]:
def create_model_2(x_train,x_val):
    weights_path = '../input/weightsclimbers/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    vgg16 = VGG16(weights=None, include_top=False, input_shape=(IMG_SIZE_X, IMG_SIZE_Y, 3))
    vgg16.load_weights(weights_path, by_name=True)
    vgg16.trainable = False # remove if you want to retrain vgg16 weights
    # vgg16.summary()
    transfer_model_2 = Sequential()
    transfer_model_2.add(vgg16)
    transfer_model_2.add(Flatten())
    transfer_model_2.add(Dense(128, activation='relu'))
    transfer_model_2.add(Dropout(0.2))
    transfer_model_2.add(Dense(5, activation='softmax'))
    transfer_model_2.summary()
    x_train_new = x_train
    x_val_new = x_val
    x_train_new = keras.applications.vgg16.preprocess_input(x_train_new)
    x_val_new = keras.applications.vgg16.preprocess_input(x_val_new)
    return transfer_model_2, x_train_new, x_val_new

In [12]:
def create_model_3(x_train,x_val):
    weights_path = '../input/weightsclimbers/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    inceptionV3 = InceptionV3(weights=None, include_top=False, input_shape=(IMG_SIZE_X, IMG_SIZE_Y, 3))
    inceptionV3.load_weights(weights_path, by_name=True)
    inceptionV3.trainable = False # remove if you want to retrain rinceptionV3 weights
    # inceptionV3.summary()
    transfer_model_3 = Sequential()
    transfer_model_3.add(inceptionV3)
    transfer_model_3.add(Flatten())
    transfer_model_3.add(Dense(128, activation='relu'))
    transfer_model_3.add(Dropout(0.2))
    transfer_model_3.add(Dense(5, activation='softmax'))
    transfer_model_3.summary()
    x_train_new = x_train
    x_val_new = x_val
    x_train_new = keras.applications.inception_v3.preprocess_input(x_train_new)
    x_val_new = keras.applications.inception_v3.preprocess_input(x_val_new)
    return transfer_model_3, x_train_new, x_val_new

In [18]:
def train_model(model, batch_size, epochs, x_train_new, x_val_new, y_train_onehot, y_val_onehot,class_weights):
    
    batch_size = batch_size
    epochs = epochs
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
              metrics=['categorical_accuracy',keras.metrics.AUC(),keras.metrics.Recall()])
    
    
    history = model.fit(x_train_new, y_train_onehot, \
                              batch_size=batch_size, epochs=epochs, \
                              validation_split=0.2, verbose=1, shuffle=True, validation_data=(x_val_new, y_val_onehot),
                       class_weight = class_weights)
    return model

In [36]:
################## Main function ######################


new_h = IMG_SIZE_X
new_w = IMG_SIZE_Y
batch_size = 500
epochs = 10
channels = 3
#Divide data into test and training/validation for three different models
X_test, Y_test, X_model_1, Y_model_1, X_model_2, Y_model_2, X_model_3, Y_model_3 = train_test_split_modified(data)
training_data = [X_model_1, Y_model_1, X_model_2, Y_model_2, X_model_3, Y_model_3]
#Check that the training datasets are correctly stratified 
Y_model_1_df = DataFrame(Y_model_1,columns=['labels'])
#print(Y_model_1_df.labels.value_counts())
Y_model_2_df = DataFrame(Y_model_2,columns=['labels'])
#print(Y_model_2_df.labels.value_counts())
Y_model_3_df = DataFrame(Y_model_3,columns=['labels'])
#print(Y_model_3_df.labels.value_counts())
#deaseases explanation: {"0": "Cassava Bacterial Blight (CBB)", 
#"1": "Cassava Brown Streak Disease (CBSD)", "2": "Cassava Green Mottle (CGM)", 
#"3": "Cassava Mosaic Disease (CMD)", "4": "Healthy"}

# #visualize each of the classes
# fig = plt.figure(figsize=(10, 6))

# for i in range(8):
#     img = X_model_1[i]
#     fig.add_subplot(2, 4, i+1)
#     plt.imshow(img)
#     plt.title(Y_model_1[i])
    
#Baseline model
x_train, x_val, y_train, y_val = data_preparation(training_data[0], training_data[1], new_h, new_w, channels)
y_train_onehot, y_val_onehot, class_weights = encode_target_variable(y_train,y_val)
input_shape = [new_h,new_w,channels]
del y_train, y_val
baseline_model = create_model_0(input_shape)
gc.collect()
baseline_model = train_model(baseline_model, batch_size, epochs, x_train, x_val, y_train_onehot, y_val_onehot, class_weights)
#Resnet50
x_train, x_val, y_train, y_val = data_preparation(training_data[0], training_data[1], new_h, new_w, channels)
y_train_onehot, y_val_onehot, class_weights = encode_target_variable(y_train,y_val)
del y_train, y_val
resnet50_model, x_train_new, x_val_new = create_model_1(x_train,x_val,input_shape)
del x_train,x_val
gc.collect()
resnet50_model = train_model(resnet50_model, batch_size, epochs, x_train_new, x_val_new, y_train_onehot, y_val_onehot, class_weights)
#vgg16
x_train, x_val, y_train, y_val = data_preparation(training_data[2], training_data[3], new_h, new_w, channels)
y_train_onehot, y_val_onehot, class_weights = encode_target_variable(y_train,y_val)
del y_train, y_val
vgg16_model, x_train_new, x_val_new = create_model_2(x_train,x_val)
del x_train,x_val
gc.collect()
vgg16_model = train_model(vgg16_model, batch_size, epochs, x_train_new, x_val_new, y_train_onehot, y_val_onehot)
#InceptionV3
x_train, x_val, y_train, y_val = data_preparation(training_data[4], training_data[5], new_h, channels)
y_train_onehot, y_val_onehot, class_weights = encode_target_variable(y_train,y_val)
del y_train, y_val
inception_model, x_train_new, x_val_new = create_model_3(x_train,x_val)
del x_train,x_val
gc.collect()
inception_model = train_model(inception_model, batch_size, epochs, x_train_new, x_val_new, y_train_onehot, y_val_onehot, class_weights)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4], y=[2 1 3 ... 3 1 0] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


{0: 3.937593984962406, 1: 1.9541044776119403, 2: 1.7934931506849314, 3: 0.32517851598882336, 4: 1.6625396825396825}
Model: "functional_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_81 (Conv2D)              (None, 128, 128, 16) 1216        input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_84 (Conv2D)              (None, 128, 128, 16) 448         input_18[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_54 (MaxPooling2D) (None, 64, 64, 16)   0           conv

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4], y=[2 1 3 ... 3 1 0] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


{0: 3.937593984962406, 1: 1.9541044776119403, 2: 1.7934931506849314, 3: 0.32517851598882336, 4: 1.6625396825396825}
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
flatten_10 (Flatten)         (None, 32768)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 128)               4194432   
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 5)                 645       
Total params: 27,782,789
Trainable params: 4,195,077
Non-trainable params: 23,587,712
____________________________________________________

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2 3 4], y=[3 2 2 ... 4 1 2] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


ValueError: too many values to unpack (expected 2)

In [ ]:

def load_test_data(new_w,new_h):
    path = '../input/cassava-leaf-disease-classification/test_images'
    path_loop = r'../input/cassava-leaf-disease-classification/test_images/*.*'
    onlyfiles = next(os.walk(path))[2] #dir is your directory path as string
    numOfFiles = len(onlyfiles)
    data = []
    name_files = []
    for file in tqdm(glob.glob(path_loop)):
        a=cv2.imread(file)
        name_file = os.path.basename(file)
        #conversion numpy array into rgb image to show
        c = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        h, w, channels = c.shape
        #input size of Resnet architecture
        frame_rgb = cv2.resize(c,(new_w,new_h),interpolation=cv2.INTER_CUBIC)
        x_train = np.array(frame_rgb) / 255
        x_train = x_train.reshape(-1, new_h, new_w, channels)
        data.append([x_train])
        name_files.append(name_file)
    return data, name_files

In [ ]:
# Create the CSV
data, name_files = load_test_data(IMG_SIZE_X,IMG_SIZE_Y)
predictions = inception_model.predict(data)
pred_df = pd.DataFrame(predictions)
pred_list = pred_df.idxmax(axis="columns")
pred_df = pd.DataFrame(zip(name_files, pred_list), columns = ["image_id", "label"])
pred_df.head()
pd.DataFrame(pred_df, columns=['predictions']).to_csv('submission.csv')